# <div style="color:white;display:fill;border-radius:15px;background-color:#032137;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:180%">Template DS</p></div>

## <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">1. Importings</p></div>

## 1.1 Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import math
import datetime

import plotly.express as px
from matplotlib            import pyplot as plt

import xgboost as xgb
from sklearn.metrics       import mean_absolute_error, mean_squared_error
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from category_encoders     import CountEncoder


In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.expand_frame_repr', False )
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
plt.rcParams['figure.figsize'] = [16, 12]
plt.rcParams['font.size'] = 24

## 1.2 Dados

In [3]:
stores_raw = pd.read_csv('./data/stores.csv')
train_raw = pd.read_csv('./data/train.csv')
train_features_raw = pd.read_csv('./data/train_features.csv')
test_raw = pd.read_csv('./data/test.csv')
test_features_raw = pd.read_csv('./data/test_features.csv')


## 1.3 Funções auxiliares

In [54]:
def tratamento_dados(df):
    """
    Faz o tratamento dos dados aplicando todas as conversões necessárias.

    Args:
        df (DataFrame): Um DataFrame de Pandas a ser modificado

    Returns:
        DataFrame: Um DataFrame de Pandas modificado.
    """
    count_encoder = CountEncoder()
    
    # Imputando os valores de tamanho
    df['tamanho'] = df['tamanho'].apply(lambda x: 39778.0 if x <= 60 else 150046.423077 if x > 70 else 67256.666667)
    
    # Imputando zeros nos descontos ausentes
    df[['desconto_1', 'desconto_2', 'desconto_3', 'desconto_4', 'desconto_5']] = df[['desconto_1', 'desconto_2', 'desconto_3', 'desconto_4', 'desconto_5']].fillna(0)
    
    # Transformando a data em datetime
    df['data'] = df['data'] + '-2023'
    df['data'] = pd.to_datetime(df['data'], format= "%m-%d-%Y")
    
    # Distrinchando colunas de data
    df['dia'] = df['data'].dt.day
    df['mes'] = df['data'].dt.month
    df['semana_do_ano'] = df['data'].dt.isocalendar().week
    df['dia_da_semana'] = df['data'].dt.day_of_week
    
    # Convertendo a coluna feriado para variável binária
    df['feriado'] = df['feriado'].apply(lambda x: 1 if x == 'sim' else 0)
    
    # Imputando distâncias para competidores enormes no lugar dos NAs
    df['distancia_competidores'] = df['distancia_competidores'].apply(lambda x: 400000.0 if math.isnan(x) else x)
    
    # Criando uma coluna com a soma dos descontos oferecidos
    df['descontos'] = df[['desconto_1', 'desconto_2', 'desconto_3', 'desconto_4', 'desconto_5']].sum(axis=1)
    
    # Criando uma coluna boolena de desconto
    df['tem_desconto'] = df['descontos'].apply(lambda x: 1 if x != 0 else 0)
    
    # dis da semana
    df['dia_da_semana_sin'] = df['dia_da_semana'].apply( lambda x: np.sin( x * ( 2. * np.pi/7 ) ) )
    df['dia_da_semana_cos'] = df['dia_da_semana'].apply( lambda x: np.cos( x * ( 2. * np.pi/7 ) ) )

    # mês
    df['mes_sin'] = df['mes'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
    df['mes_cos'] = df['mes'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

    # dia 
    df['dia_sin'] = df['dia'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) )
    df['dia_cos'] = df['dia'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )

    # semana_do_ano
    df['semana_do_ano_sin'] = df['semana_do_ano'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
    df['semana_do_ano_cos'] = df['semana_do_ano'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )
    
    # Encoding
    df['tipo'] = count_encoder.fit_transform(df['tipo'])
    
    return df

def cross_validation( x_training, kfold, model_name, model, verbose=False ):
    mae_list = []
    mape_list = []
    rmse_list = []
    for k in reversed( range( 1, kfold+1 ) ):
        if verbose:
            print( '\nKFold Number: {}'.format( k ) )
        # start and end date for validation 
        validation_start_date = x_training['data'].max() - datetime.timedelta( days=k*5*7)
        validation_end_date = x_training['data'].max() - datetime.timedelta( days=(k-1)*5*7)

        # filtering dataset
        training = x_training[x_training['data'] < validation_start_date]
        validation = x_training[(x_training['data'] >= validation_start_date) & (x_training['data'] <= validation_end_date)]

        # training and validation dataset
        # training
        xtraining = training[selected_columns]
        ytraining = training['vendas_semanais']

        # validation
        xvalidation = validation[selected_columns]
        yvalidation = validation['vendas_semanais']

        # model
        m = model.fit( xtraining, ytraining )

        # prediction
        yhat = m.predict( xvalidation )

        # performance
        m_result = ml_error( model_name, yvalidation, yhat )

        # store performance of each kfold iteration
        mae_list.append(  m_result['MAE'] )
        mape_list.append( m_result['MAPE'] )
        rmse_list.append( m_result['RMSE'] )

    return pd.DataFrame( {'Model Name': model_name,
                          'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
                          'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
                          'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str ) }, index=[0] )


def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )
     
    
def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( ( y - yhat ) / y ) )

def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( { 'Model Name': model_name, 
                           'MAE': mae, 
                           'MAPE': mape,
                           'RMSE': rmse }, index=[0] )

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">2. Data exploration and problem comprehension</p></div>


- Main goal/problem
- Sub-goals
- What will the finished product be?

## 2.1 Primeiras explorações

### Stores

In [5]:
stores_raw.shape

(45, 3)

In [6]:
stores_raw.isna().sum()

loja        0
tipo        0
tamanho    11
dtype: int64

In [7]:
stores_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   loja     45 non-null     int64  
 1   tipo     45 non-null     object 
 2   tamanho  34 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ KB


In [8]:
stores_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
loja,45.0,23.0,13.133926,1.0,12.00,23.0,34.00,45.0
tamanho,34.0,126525.5,60178.808984,34875.0,76331.75,126172.5,177610.25,219622.0


### Train


In [9]:
train_raw.shape


(135385, 6)

In [10]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135385 entries, 0 to 135384
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               135385 non-null  int64  
 1   loja             135385 non-null  int64  
 2   setor            135385 non-null  int64  
 3   data             135385 non-null  object 
 4   vendas_semanais  122523 non-null  float64
 5   feriado          135385 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


### Train_features

In [11]:
train_features_raw.shape

(2070, 13)

In [12]:
train_features_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   loja                    2070 non-null   int64  
 1   data                    2070 non-null   object 
 2   temperatura             1336 non-null   float64
 3   combustivel             1262 non-null   float64
 4   desconto_1              90 non-null     float64
 5   desconto_2              89 non-null     float64
 6   desconto_3              84 non-null     float64
 7   desconto_4              75 non-null     float64
 8   desconto_5              90 non-null     float64
 9   desemprego              2070 non-null   float64
 10  feriado                 2070 non-null   object 
 11  distancia_competidores  90 non-null     float64
 12  clientes                2070 non-null   int64  
dtypes: float64(9), int64(2), object(2)
memory usage: 210.4+ KB


In [13]:
train_features_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
loja,2070.0,23.000000,12.990311,1.000000,12.000000,23.000000,34.000000,45.000000
temperatura,1336.0,16.047276,10.730922,-18.922222,8.711111,17.538889,23.668056,37.588889
combustivel,1262.0,3.596067,0.286606,2.891000,3.452000,3.623000,3.806500,4.211000
desconto_1,90.0,8365.174333,7969.138443,5.640000,3224.867500,6119.695000,10845.145000,34348.140000
desconto_2,89.0,4888.274494,7607.164623,2.630000,247.290000,1090.920000,7331.950000,44021.610000
desconto_3,84.0,212.091667,232.489814,1.320000,57.027500,141.970000,277.010000,1134.490000
desconto_4,75.0,2446.924400,3081.045660,38.350000,670.685000,1292.580000,3180.255000,20834.370000
desconto_5,90.0,5297.985556,3756.863013,578.020000,3062.557500,4614.910000,6522.022500,27754.230000
desemprego,2070.0,0.081183,0.018841,0.044200,0.071930,0.078660,0.085490,0.140210
distancia_competidores,90.0,16933.288889,14847.975897,1576.000000,6071.000000,12023.500000,21748.500000,68224.000000


### Test


In [14]:
test_raw.shape


(18068, 5)

In [15]:
test_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18068 entries, 0 to 18067
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       18068 non-null  int64 
 1   loja     18068 non-null  int64 
 2   setor    18068 non-null  int64 
 3   data     18068 non-null  object
 4   feriado  18068 non-null  object
dtypes: int64(3), object(2)
memory usage: 705.9+ KB


In [16]:
test_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
id,18068.0,285097.500000,5215.926667,276064.0,280580.75,285097.5,289614.25,294131.0
loja,18068.0,22.173899,12.788277,1.0,11.00,22.0,33.00,45.0
setor,18068.0,44.723268,30.643778,1.0,18.00,38.0,74.00,99.0


### Test_features


In [17]:
test_features_raw.shape


(270, 13)

In [18]:
test_features_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   loja                    270 non-null    int64  
 1   data                    270 non-null    object 
 2   temperatura             168 non-null    float64
 3   combustivel             174 non-null    float64
 4   desconto_1              264 non-null    float64
 5   desconto_2              204 non-null    float64
 6   desconto_3              258 non-null    float64
 7   desconto_4              227 non-null    float64
 8   desconto_5              270 non-null    float64
 9   desemprego              270 non-null    float64
 10  feriado                 270 non-null    object 
 11  distancia_competidores  264 non-null    float64
 12  clientes                270 non-null    int64  
dtypes: float64(9), int64(2), object(2)
memory usage: 27.5+ KB


### Cópia de segurança

In [19]:
stores = stores_raw.copy()
train = train_raw.copy()
train_features = train_features_raw.copy()
test = test_raw.copy()
test_features = test_features_raw.copy()

## 2.2 Fundindo os dataframes

### Train


In [20]:
df_train_merged = train.merge(stores, how='left', on='loja')
df_train_merged = df_train_merged.merge(train_features, how='left', on=['loja', 'data', 'feriado'])
df_train_merged.sample(20)

,id,loja,setor,data,vendas_semanais,feriado,tipo,tamanho,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes
12023,152702,2,52,02-04,2873.49,nao,eletrodomestico,202307.0,NaN,2.989,NaN,NaN,NaN,NaN,NaN,0.08028,NaN,79
101229,241908,45,35,09-02,1540.44,nao,eletronico,NaN,21.461111,NaN,NaN,NaN,NaN,NaN,NaN,0.08625,NaN,437
117117,257796,28,98,10-07,13347.00,nao,eletrodomestico,206302.0,21.355556,3.827,NaN,NaN,NaN,NaN,NaN,0.12890,NaN,533
126529,267208,16,24,11-04,3321.77,nao,eletronico,57197.0,NaN,3.527,NaN,NaN,NaN,NaN,NaN,0.06232,NaN,260
116772,257451,21,54,10-07,53.72,nao,eletronico,140167.0,22.022222,NaN,NaN,NaN,NaN,NaN,NaN,0.07441,NaN,131
30488,171167,30,72,03-18,28.85,nao,outro,NaN,17.366667,NaN,NaN,NaN,NaN,NaN,NaN,0.08028,NaN,732
67581,208260,3,34,06-10,4706.39,nao,eletronico,NaN,29.355556,3.648,NaN,NaN,NaN,NaN,NaN,0.07574,NaN,517
118983,259662,19,49,10-14,2167.40,nao,eletrodomestico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.07866,NaN,610
18181,158860,6,41,02-18,3324.00,nao,eletrodomestico,202505.0,15.338889,NaN,NaN,NaN,NaN,NaN,NaN,0.06858,NaN,395
19391,160070,26,55,02-18,10558.02,nao,eletrodomestico,152513.0,NaN,3.263,NaN,NaN,NaN,NaN,NaN,0.07907,NaN,153


### Test


In [21]:
df_test_merged = test.merge(stores, how='left', on='loja')
df_test_merged = df_test_merged.merge(test_features, how='left', on=['loja', 'data', 'feriado'])
df_test_merged.sample(20)

,id,loja,setor,data,feriado,tipo,tamanho,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes
15377,291441,5,16,12-30,sim,eletronico,34875.0,7.566667,3.129,1938.28,17079.76,72.93,162.33,1211.95,0.06300,7665.0,344
1791,277855,9,4,11-25,sim,eletronico,125833.0,12.400000,NaN,121.57,NaN,44061.13,247.83,8294.47,0.06054,3051.0,219
11489,287553,40,81,12-16,nao,eletrodomestico,155083.0,NaN,NaN,3067.81,NaN,383.98,409.64,2938.11,0.04420,8198.0,26
8768,284832,31,34,12-09,nao,eletrodomestico,203750.0,NaN,3.158,2876.83,11.87,471.16,1915.49,11604.37,0.07441,851.0,622
3015,279079,38,94,11-25,sim,outro,39690.0,11.805556,NaN,17.17,NaN,5.26,NaN,615.68,0.12890,4419.0,640
15379,291443,5,94,12-30,sim,eletronico,34875.0,7.566667,3.129,1938.28,17079.76,72.93,162.33,1211.95,0.06300,7665.0,344
4759,280823,13,35,12-02,nao,eletrodomestico,219622.0,1.077778,3.389,7945.42,56.00,25126.90,5550.87,31844.20,0.06392,10953.0,943
3778,279842,22,49,12-02,nao,eletronico,119557.0,NaN,3.501,10739.64,NaN,926.79,3397.08,29944.71,0.07706,3465.0,173
3442,279506,32,6,12-02,nao,eletrodomestico,203007.0,1.000000,NaN,5261.51,65.00,1075.86,4742.55,25961.21,0.08513,10438.0,120
6228,282292,21,29,12-09,nao,eletronico,140167.0,NaN,NaN,6862.30,19.00,330.90,4746.58,17822.36,0.07441,902.0,637


### Re-inspecionando o conjunto de treino

In [22]:
df_train_merged.describe().T

,count,mean,std,min,25%,50%,75%,max
id,135385.0,208371.000000,39082.427434,140679.000000,174525.000000,208371.000000,242217.000000,276063.000000
loja,135385.0,22.210681,12.783303,1.000000,11.000000,22.000000,33.000000,45.000000
setor,135385.0,44.150327,30.451533,1.000000,18.000000,37.000000,72.000000,99.000000
vendas_semanais,122523.0,62991.392230,198445.286559,-1321.480000,2228.580000,8236.320000,23589.140000,999967.424022
tamanho,102058.0,132619.110310,57153.086978,34875.000000,103681.000000,128107.000000,196321.000000,219622.000000
temperatura,87440.0,15.733733,10.735085,-18.922222,8.244444,17.283333,23.388889,37.588889
combustivel,82396.0,3.597816,0.285991,2.891000,3.459000,3.623000,3.807000,4.211000
desconto_1,5926.0,8883.068046,7933.224879,5.640000,3648.400000,6756.030000,10941.050000,34348.140000
desconto_2,5875.0,5213.345593,7754.955048,2.630000,274.850000,1229.960000,7898.330000,44021.610000
desconto_3,5639.0,221.416875,232.376139,1.320000,61.080000,143.880000,292.440000,1134.490000


In [23]:
df_train_merged = df_train_merged[df_train_merged['vendas_semanais']>0]

### Obtendo o número de setores e o tamanho de cada loja

In [24]:

aux = df_train_merged[['loja', 'setor', 'tamanho']].groupby('loja').agg({'setor':'nunique', 'tamanho':'mean'}).reset_index()
aux.head()

,loja,setor,tamanho
0,1,75,151315.0
1,2,75,202307.0
2,3,69,NaN
3,4,74,NaN
4,5,69,34875.0


## 2.3 Fazendo o primeiro tratamento de dados no conjunto de treino


In [25]:
# print(aux[aux['setor']<= 60][['tamanho']].mean())
# print(aux[(aux['setor'] > 60) & (aux['setor'] <= 70)][['tamanho']].mean())
# print(aux[aux['setor']> 70][['tamanho']].mean())

In [26]:
# df_train_merged['tamanho'] = df_train_merged['tamanho'].apply(lambda x: 39778.0 if x <= 60 else 150046.423077 if x > 70 else 67256.666667)

In [27]:
# df_train_merged[['desconto_1', 'desconto_2', 'desconto_3', 'desconto_4', 'desconto_5']] = df_train_merged[['desconto_1', 'desconto_2', 'desconto_3', 'desconto_4', 'desconto_5']].fillna(0)
# df_train_merged['data'] = df_train_merged['data'] + '-2023'
# df_train_merged['data'] = pd.to_datetime(df_train_merged['data'], format= "%m-%d-%Y")
# df_train_merged['feriado'] = df_train_merged['feriado'].apply(lambda x: 1 if x == 'sim' else 0)
# df_train_merged['distancia_competidores'] = df_train_merged['distancia_competidores'].apply(lambda x: 400000.0 if math.isnan(x) else x)

In [28]:
# df_train_merged['descontos'] = df_train_merged[['desconto_1', 'desconto_2', 'desconto_3', 'desconto_4', 'desconto_5']].sum(axis=1)

In [29]:
# df_train_merged['dia'] = df_train_merged['data'].dt.day
# df_train_merged['mes'] = df_train_merged['data'].dt.month
# df_train_merged['semana_do_ano'] = df_train_merged['data'].dt.isocalendar().week
# df_train_merged['ano_semana'] = df_train_merged['data'].dt.strftime('%Y-%U')

In [30]:
df_train = tratamento_dados(df_train_merged)

In [31]:
df_train.isna().sum()

id                            0
loja                          0
setor                         0
data                          0
vendas_semanais               0
feriado                       0
tipo                          0
tamanho                       0
temperatura               43194
combustivel               47830
desconto_1                    0
desconto_2                    0
desconto_3                    0
desconto_4                    0
desconto_5                    0
desemprego                    0
distancia_competidores        0
clientes                      0
dia                           0
mes                           0
semana_do_ano                 0
dia_da_semana                 0
descontos                     0
tem_desconto                  0
dia_da_semana_sin             0
dia_da_semana_cos             0
mes_sin                       0
mes_cos                       0
dia_sin                       0
dia_cos                       0
semana_do_ano_sin             0
semana_d

## 2.4 Fazendo o primeiro tratamento de dados no conjunto de teste

In [32]:
df_test = tratamento_dados(df_test_merged)

In [33]:
df_test

,id,loja,setor,data,feriado,tipo,tamanho,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes,dia,mes,semana_do_ano,dia_da_semana,descontos,tem_desconto,dia_da_semana_sin,dia_da_semana_cos,mes_sin,mes_cos,dia_sin,dia_cos,semana_do_ano_sin,semana_do_ano_cos
0,276064,20,16,2023-11-25,1,9204,67256.666667,7.988889,3.492,335.66,80.00,101378.79,64.46,2251.98,0.07082,3419.0,143,25,11,47,5,104110.89,1,-0.974928,-0.222521,-5.000000e-01,0.866025,-8.660254e-01,0.5,-5.680647e-01,0.822984
1,276065,39,14,2023-11-25,1,9204,150046.423077,19.088889,3.236,224.08,292.94,77126.16,77.74,4875.43,0.07716,573.0,516,25,11,47,5,82596.35,1,-0.974928,-0.222521,-5.000000e-01,0.866025,-8.660254e-01,0.5,-5.680647e-01,0.822984
2,276066,40,5,2023-11-25,1,9204,150046.423077,0.422222,3.536,247.58,387.88,40362.07,47.00,1788.31,0.04420,3707.0,227,25,11,47,5,42832.84,1,-0.974928,-0.222521,-5.000000e-01,0.866025,-8.660254e-01,0.5,-5.680647e-01,0.822984
3,276067,24,92,2023-11-25,1,9204,67256.666667,5.461111,3.689,2571.98,66.94,64304.51,221.93,3661.62,0.08454,6482.0,533,25,11,47,5,70826.98,1,-0.974928,-0.222521,-5.000000e-01,0.866025,-8.660254e-01,0.5,-5.680647e-01,0.822984
4,276068,22,20,2023-11-25,1,7034,150046.423077,7.933333,NaN,1649.31,0.00,70087.94,7.50,5266.90,0.07706,3032.0,551,25,11,47,5,77011.65,1,-0.974928,-0.222521,-5.000000e-01,0.866025,-8.660254e-01,0.5,-5.680647e-01,0.822984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18063,294127,11,94,2023-12-30,1,9204,67256.666667,NaN,NaN,4241.32,58046.41,239.33,78.09,586.72,0.07197,12896.0,302,30,12,52,5,63191.87,1,-0.974928,-0.222521,-2.449294e-16,1.000000,-2.449294e-16,1.0,-2.449294e-16,1.000000
18064,294128,15,18,2023-12-30,1,7034,150046.423077,-0.311111,3.566,3248.40,31122.20,111.35,605.88,3474.84,0.07866,8022.0,363,30,12,52,5,38562.67,1,-0.974928,-0.222521,-2.449294e-16,1.000000,-2.449294e-16,1.0,-2.449294e-16,1.000000
18065,294129,11,30,2023-12-30,1,9204,67256.666667,NaN,NaN,4241.32,58046.41,239.33,78.09,586.72,0.07197,12896.0,302,30,12,52,5,63191.87,1,-0.974928,-0.222521,-2.449294e-16,1.000000,-2.449294e-16,1.0,-2.449294e-16,1.000000
18066,294130,37,46,2023-12-30,1,1830,67256.666667,9.033333,NaN,373.92,1057.77,1.50,10.08,741.34,0.07716,5006.0,553,30,12,52,5,2184.61,1,-0.974928,-0.222521,-2.449294e-16,1.000000,-2.449294e-16,1.0,-2.449294e-16,1.000000


## 2.5 Removendo os NAs sobressalentes

In [34]:
df_train.isna().sum()

id                            0
loja                          0
setor                         0
data                          0
vendas_semanais               0
feriado                       0
tipo                          0
tamanho                       0
temperatura               43194
combustivel               47830
desconto_1                    0
desconto_2                    0
desconto_3                    0
desconto_4                    0
desconto_5                    0
desemprego                    0
distancia_competidores        0
clientes                      0
dia                           0
mes                           0
semana_do_ano                 0
dia_da_semana                 0
descontos                     0
tem_desconto                  0
dia_da_semana_sin             0
dia_da_semana_cos             0
mes_sin                       0
mes_cos                       0
dia_sin                       0
dia_cos                       0
semana_do_ano_sin             0
semana_d

In [35]:
df_train.dropna(axis=0, inplace=True)
df_train

,id,loja,setor,data,vendas_semanais,feriado,tipo,tamanho,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes,dia,mes,semana_do_ano,dia_da_semana,descontos,tem_desconto,dia_da_semana_sin,dia_da_semana_cos,mes_sin,mes_cos,dia_sin,dia_cos,semana_do_ano_sin,semana_do_ano_cos
0,140679,17,93,2023-01-07,6283.00,0,47226,150046.423077,-14.316667,2.891,0.00,0.00,0.00,0.00,0.00,0.06866,400000.0,541,7,1,1,5,0.00,0,-0.974928,-0.222521,0.5,0.866025,0.994522,0.104528,0.120537,0.992709
1,140680,12,17,2023-01-07,10006.77,0,47226,150046.423077,3.133333,3.287,0.00,0.00,0.00,0.00,0.00,0.14021,400000.0,463,7,1,1,5,0.00,0,-0.974928,-0.222521,0.5,0.866025,0.994522,0.104528,0.120537,0.992709
4,140683,26,9,2023-01-07,9526.27,0,62487,150046.423077,-6.038889,3.193,0.00,0.00,0.00,0.00,0.00,0.07907,400000.0,124,7,1,1,5,0.00,0,-0.974928,-0.222521,0.5,0.866025,0.994522,0.104528,0.120537,0.992709
6,140685,34,72,2023-01-07,36338.44,0,62487,150046.423077,-4.166667,2.980,0.00,0.00,0.00,0.00,0.00,0.10398,400000.0,395,7,1,1,5,0.00,0,-0.974928,-0.222521,0.5,0.866025,0.994522,0.104528,0.120537,0.992709
9,140688,2,36,2023-01-07,766.00,0,62487,150046.423077,7.050000,2.976,0.00,0.00,0.00,0.00,0.00,0.08028,400000.0,910,7,1,1,5,0.00,0,-0.974928,-0.222521,0.5,0.866025,0.994522,0.104528,0.120537,0.992709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135337,276016,17,97,2023-11-18,7145.60,0,47226,150046.423077,0.516667,3.489,6004.88,247.29,32.39,653.49,4391.01,0.06617,12619.0,931,18,11,46,5,11329.06,1,-0.974928,-0.222521,-0.5,0.866025,-0.587785,-0.809017,-0.663123,0.748511
135340,276019,17,7,2023-11-18,15465.86,0,47226,150046.423077,0.516667,3.489,6004.88,247.29,32.39,653.49,4391.01,0.06617,12619.0,931,18,11,46,5,11329.06,1,-0.974928,-0.222521,-0.5,0.866025,-0.587785,-0.809017,-0.663123,0.748511
135353,276032,17,95,2023-11-18,49004.50,0,47226,150046.423077,0.516667,3.489,6004.88,247.29,32.39,653.49,4391.01,0.06617,12619.0,931,18,11,46,5,11329.06,1,-0.974928,-0.222521,-0.5,0.866025,-0.587785,-0.809017,-0.663123,0.748511
135360,276039,15,90,2023-11-18,5815.04,0,47226,150046.423077,8.072222,3.717,3668.40,615.27,49.57,946.20,4393.97,0.07866,7943.0,314,18,11,46,5,9673.41,1,-0.974928,-0.222521,-0.5,0.866025,-0.587785,-0.809017,-0.663123,0.748511


# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">3. Hypothesis Mental Map Creation</p></div>


- Mental map for hypothesis and questions
- Hypothesis and questions list

In [36]:
# H1: As vendas estão correlacionadas positivamente com o tamanho da loja.

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">4. Feature Engineering</p></div>


- Fillout remaining NAs 
- Derive new variables as needed

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">5. Data selection and filtering</p></div>



- Filter data rows
- Filter data columns
- Based on the questions and hypothesis, select columns
- Create a new filtered dataframe
- Create the widgets to filter the data

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">6. Exploratory Data Analysis (EDA)</p></div>


- Answer the hypothesis list
- Build data visualization solutions and plots

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">7. Data Preparation</p></div>


- Normalize, re-scale and transform (enconding) variables to suit model requirements
- It may be a good idea to normalize all of the features so they are comparable in magnitude

## 7.1 Encodings

### 7.2 Splitando os dados para fazer validação

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">8. Feature Selection through Boruta algorithm</p></div>


- Use Boruta algorithm to select best features to machine learning models

In [37]:
# # Separando em treino e validação
# train = df_train[(df_train['data'] >= pd.to_datetime('2023-01-01', format='%Y-%m-%d')) & (df_train['data'] <= pd.to_datetime('2023-10-20', format='%Y-%m-%d'))]
# validation = df_train[(df_train['data'] > pd.to_datetime('2023-10-20', format='%Y-%m-%d')) & (df_train['data'] <= pd.to_datetime('2023-11-23', format='%Y-%m-%d'))]

In [38]:
# print(f"O conjunto de treino tem {train.shape[0]} linhas.")
# print(f"O conjunto de validação tem {validation.shape[0]} linhas.")

In [39]:
df_train.columns

Index(['id', 'loja', 'setor', 'data', 'vendas_semanais', 'feriado', 'tipo',
       'tamanho', 'temperatura', 'combustivel', 'desconto_1', 'desconto_2',
       'desconto_3', 'desconto_4', 'desconto_5', 'desemprego',
       'distancia_competidores', 'clientes', 'dia', 'mes', 'semana_do_ano',
       'dia_da_semana', 'descontos', 'tem_desconto', 'dia_da_semana_sin',
       'dia_da_semana_cos', 'mes_sin', 'mes_cos', 'dia_sin', 'dia_cos',
       'semana_do_ano_sin', 'semana_do_ano_cos'],
      dtype='object')

In [49]:
selected_columns = ['id', 'loja', 'setor', 'feriado',
       'tamanho', 'desemprego',
       'distancia_competidores', 'clientes', 'dia', 'mes', 'semana_do_ano',
       'dia_da_semana', 'descontos', 'tem_desconto',
       'dia_da_semana_sin', 'dia_da_semana_cos', 'mes_sin', 'mes_cos',
       'dia_sin', 'dia_cos', 'semana_do_ano_sin', 'semana_do_ano_cos'] #'tipo', 'vendas_semanais', 'data', 'temperatura', 'combustivel'

selected_columns_with_data = selected_columns.copy()
selected_columns_with_data.extend([ 'data', 'vendas_semanais' ])

In [51]:
X_train = df_train[selected_columns]
y_train = df_train['vendas_semanais']

X_training = df_train[selected_columns_with_data]

# X_val = validation[selected_columns]
# y_val = validation['vendas_semanais']

X_test = df_test[selected_columns]

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">9. Model Implementation</p></div>


- Implement different machine learning models and algorithms
- Conduct cross-velidation computing
- Conduct single performance metrics computing

## 9.0 Cross-validation

## 9.1 XGBoostRegressor

In [52]:
# model
model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
                              n_estimators=100, 
                              eta=0.01, 
                              max_depth=10, 
                              subsample=0.7,
                              colsample_bytree=0.9 ).fit( X_train, y_train )

# # prediction
# yhat_val_xgb = model_xgb.predict( X_val )

# # performance
# xgb_result = ml_error( 'XGBoost Regressor', y=y_val, yhat=yhat_val_xgb)
# xgb_result

In [44]:
# yhat_val_xgb

In [55]:
xgb_result_cv = cross_validation(x_training=X_training, kfold=5, model_name='XGBoost', model=model_xgb, verbose=False )
xgb_result_cv

,Model Name,MAE CV,MAPE CV,RMSE CV
0,XGBoost,96189.2 +/- 3388.66,1210.44 +/- 1187.5,197194.3 +/- 4153.52


## 9.2 Regressão Linear

## 9.3 Retreinando os modelos com todos os dados de treino

In [ ]:
# X_train = df_train[selected_columns]
# y_train = df_train['vendas_semanais']

# X_test = df_test[selected_columns]

In [ ]:
# # model
# model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
#                               n_estimators=100, 
#                               eta=0.01, 
#                               max_depth=10, 
#                               subsample=0.7,
#                               colsample_bytree=0.9 ).fit( X_train, y_train )

# # prediction
# yhat_xgb = model_xgb.predict( X_test )

# resultados = pd.DataFrame({'id': X_test['id'], 'vendas_semanais':yhat_xgb})

In [ ]:
# resultados

,id,vendas_semanais
0,276064,63093.894531
1,276065,60878.496094
2,276066,53977.207031
3,276067,83812.460938
4,276068,90778.718750
...,...,...
18063,294127,99049.820312
18064,294128,53572.351562
18065,294129,43186.343750
18066,294130,78170.859375


In [ ]:
# resultados.to_csv('./resultados/submission.csv', index=False)

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">10. Hyperparameter Fine-Tuning</p></div>


- Implement hyperparameter search (i.e. Bayes Search) to find best model hyperparameter values
- Re-train model using best values

In [ ]:
# param_tuned = {
#     'n_estimators': 3000,
#     'eta': 0.03,
#     'max_depth': 5,
#     'subsample': 0.7,
#     'colsample_bytree': 0.7,
#     'min_child_weight': 3 
#         }

In [ ]:
# # model
# model_xgb_tuned = xgb.XGBRegressor( objective='reg:squarederror',
#                                     n_estimators=param_tuned['n_estimators'], 
#                                     eta=param_tuned['eta'], 
#                                     max_depth=param_tuned['max_depth'], 
#                                     subsample=param_tuned['subsample'],
#                                     colsample_bytree=param_tuned['colsample_bytree'],
#                                     min_child_weight=param_tuned['min_child_weight'] ).fit( X_train, y_train )

# # prediction
# yhat_xgb_tuned = model_xgb_tuned.predict( X_test )

In [ ]:
# resultados_tuned = pd.DataFrame({'id': X_test['id'], 'vendas_semanais':yhat_xgb_tuned})

In [ ]:
# resultados_tuned.to_csv('./resultados/submission.csv', index=False)

# <div style="color:white;display:fill;border-radius:15px;background-color:#123752;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:110%">11. Model Error Estimation and Interpretation</p></div>


- Use model errors to interpret the goals 
- Model learning performance
- Model generalization performance
- What it means to business?